In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/test.parquet')

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(7683577, 6)


,session,aid,ts,type,hour,day
0,11098528,11830,1661119200,0,79200,338400
1,11098529,1105029,1661119200,0,79200,338400
2,11098530,264500,1661119200,0,79200,338400
3,11098530,264500,1661119288,0,79288,338488
4,11098530,409236,1661119369,0,79369,338569


In [3]:
df.loc[df.aid==409236]

,session,aid,ts,type,hour,day
4,11098530,409236,1661119369,0,79369,338569
5,11098530,409236,1661119441,0,79441,338641
6,11098530,409236,1661120165,0,80165,339365
7,11098530,409236,1661120532,1,80532,339732
28262,11103248,409236,1661172885,0,46485,392085
...,...,...,...,...,...,...
7618612,12879442,409236,1661717329,0,72529,331729
7618613,12879442,409236,1661717490,1,72690,331890
7662079,12892431,409236,1661720610,0,75810,335010
7662080,12892431,409236,1661720638,1,75838,335038


In [4]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [5]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
0,11098528,11830,1661119200,0,79200,338400,<NA>,<NA>,0,0
1,11098529,1105029,1661119200,0,79200,338400,<NA>,<NA>,0,0
2,11098530,264500,1661119200,0,79200,338400,<NA>,0,0,0
3,11098530,264500,1661119288,0,79288,338488,0,-144736,0,0
4,11098530,409236,1661119369,0,79369,338569,144736,0,0,0


In [6]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [7]:
tmp.head()

aid
1249360    1.0
1395202    1.0
753123     1.0
1463868    1.0
478378     1.0
Name: n, dtype: float64

In [8]:
tmp.max(), tmp2.max()

(7.0, 55.0)

In [9]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

106

In [10]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
21976,1,4,0,0,-1.0,-1.0
21980,0,1,0,0,-1.0,-1.0
21981,0,0,0,0,-1.0,-1.0
21983,1,0,0,0,-1.0,-1.0
21985,2,2,0,0,-1.0,-1.0


In [11]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [12]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,5,4,36846.200000,347886.200000
2,0.000000,4,3,46810.750000,370810.750000
3,0.095679,324,184,50507.845679,261707.845679
4,0.000000,8,6,45118.750000,250318.750000
11,0.250000,4,2,45407.500000,499007.500000


In [13]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [14]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
568753,0.000000,3100.663330,64194.687500
1148262,0.577350,7549.146484,107315.273438
268905,0.447214,24479.208984,164461.546875
583066,-1.000000,-1.000000,-1.000000
1618893,0.408248,7926.070312,71733.171875


In [15]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [16]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [17]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,5,4,36846.199219,347886.18750,1.250000,0.000000,27493.654297,184398.046875,5,5,0,0,-1.0,-1.000000
2,0.000000,4,3,46810.750000,370810.75000,1.333333,0.000000,10630.142578,113783.429688,2,4,0,0,-1.0,-1.000000
3,0.095679,324,184,50507.843750,261707.84375,1.760870,0.314923,16259.557617,118062.039062,46,47,2,27,1.0,1.421053
4,0.000000,8,6,45118.750000,250318.75000,1.333333,0.000000,24547.664062,169028.281250,6,6,0,0,-1.0,-1.000000
11,0.250000,4,2,45407.500000,499007.50000,2.000000,0.500000,3890.879883,125711.054688,4,3,0,1,-1.0,1.000000


In [18]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [19]:
item_features.to_parquet('../../data/item_user_features/item10.pqt')